# Notebook for Kerk Phillips (Problem Set 1)

Contains all the coding exercises. The analytical derivations have been put in the other files, DSGE-1, DSGE-2.

In [9]:
import numpy as np
import pandas as pd
import scipy.optimize as opt
import scipy.stats as stats
import pandas as pd
import sympy as sy
from sympy.solvers import solve
from collections import namedtuple

# Exercise 5 (See the derivations in the DSGE HW-1)

In [34]:
params = ["a", "alpha", "delta","beta",  "gamma", "rho", "sigma", "tau", "zbar", "xi"]
pill = namedtuple("pill", params)
m = pill(0.50, 0.40, 0.10, 0.98, 2.50, 0.95, 0.02, 0.05, 0, 1.5)

In [35]:
## The Euler Equation

def euler_eqn(m, x):
    l = 1 #(Initialize)
    k = x
    w = (1 - m.alpha) * np.exp(m.zbar) * ((k / np.exp(m.zbar)) ** m.alpha)
    r = m.alpha * ((np.exp(m.zbar) / k) ** (1 - m.alpha))
    T = m.tau * (w + (r - m.delta)* k)
    c = (1 - m.tau) * (w + (r - m.delta)* k) + k + T - k
    euler = 1/(c ** m.gamma) - (m.beta / (c ** m.gamma)) * ((r - m.delta) * (1 - m.tau) + 1)
    return euler

## The Solver
    
def solve_ss(m):
    solve_me = lambda x: euler_eqn(m, x)
    sol = opt.root(solve_me, .5)
    
    return sol.x

In [36]:
kbar = solve_ss(m)
rbar = m.alpha * ((np.exp(m.zbar) / kbar) ** (1 - m.alpha))
wbar = (1 - m.alpha) * np.exp(m.zbar) * ((kbar / np.exp(m.zbar)) ** m.alpha)
Tbar = m.tau * (wbar + (rbar - m.delta)* kbar)
cbar = (1 - m.tau) * (wbar + (rbar - m.delta)* kbar) + Tbar

Ybar = kbar ** m.alpha * np.exp(m.zbar) ** (1 - m.alpha)
ibar = m.delta * kbar

SS_vals = {'Kbar': kbar, 'cbar': cbar, 'wbar': wbar, 'ibar': ibar, 'rbar': rbar, 'Tbar': Tbar, 
           'Ybar': Ybar}

print(SS_vals)

{'Kbar': array([7.28749795]), 'cbar': array([1.48450482]), 'wbar': array([1.32795277]), 'ibar': array([0.7287498]), 'rbar': array([0.12148228]), 'Tbar': array([0.07422524]), 'Ybar': array([2.21325461])}


In [37]:
ralg = (1 - m.beta) / (m.beta * (1 - m.tau)) + m.delta
kalg = (ralg / m.alpha) ** (1 / (m.alpha - 1))
walg = (1 - m.alpha) * (kalg) ** m.alpha
Talg = m.tau * (walg + (ralg - m.delta)* kalg)
calg = (1 - m.tau) * (walg + (ralg - m.delta)* kalg) + Talg
Yalg = kalg ** m.alpha * np.exp(m.zbar) ** (1 - m.alpha)
ialg = m.delta * kalg

SS_vals_alg = {'Kbar': kalg, 'cbar': calg, 
               'rbar': ralg, 'Tbar': Talg, 'wbar': walg,
               'Ybar': Yalg, 'ibar': ialg}

print(SS_vals_alg)

{'Kbar': 7.287497950692988, 'cbar': 1.484504818849544, 'rbar': 0.1214822771213749, 'Tbar': 0.0742252409424772, 'wbar': 1.3279527683513057, 'Ybar': 2.213254613918843, 'ibar': 0.7287497950692988}


We now lay the two values side by side in the table and compare them. Amazing similarity wohoo!

In [29]:
pd.DataFrame(SS_vals_alg, SS_vals)

,Kbar,cbar,rbar,Tbar,wbar,Ybar,ibar
Kbar,0.004345,0.109294,10.453684,0.005465,0.068131,0.279292,0.004258
cbar,0.004345,0.109294,10.453684,0.005465,0.068131,0.279292,0.004258
wbar,0.004345,0.109294,10.453684,0.005465,0.068131,0.279292,0.004258
ibar,0.004345,0.109294,10.453684,0.005465,0.068131,0.279292,0.004258
rbar,0.004345,0.109294,10.453684,0.005465,0.068131,0.279292,0.004258
Tbar,0.004345,0.109294,10.453684,0.005465,0.068131,0.279292,0.004258
Ybar,0.004345,0.109294,10.453684,0.005465,0.068131,0.279292,0.004258


# Exercise 6 (See the derivations in the DSGE HW-1)

In [30]:
def u(m, c, l):
    uc = (c**(1.0 - m.gamma) - 1.0) / (1.0 - m.gamma)
    ul = m.a * ((1.0 - l)**(1.0 - m.xi) - 1.0) / (1.0 - m.xi)
    return uc + ul

def du_dl(m, c, l):
    return - m.a * (1.0 - l)**(-m.xi)

def du_dc(m, c, l):
    return c**(-m.gamma)

def f(m, k, l, z):
    return k**m.alpha * (l * np.exp(z))**(1.0 - m.alpha)

def df_dk(m, k, l, z):
    return m.alpha * (l/k * np.exp(z))**(1.0 - m.alpha)

def df_dl(m, k, l, z):
    return (1.0 - m.alpha) * np.exp(z)**(1.0 - m.alpha) * (k / l)**m.alpha

In [31]:
def bas(m, ktp1, kt, lt, zt):
    wt = df_dl(m, kt, lt, zt)
    rt = df_dk(m, kt, lt, zt)
    Tt = m.tau * (wt*lt + (rt - m.delta)*kt)
    ct = (1 - m.tau) * (wt*lt + (rt - m.delta)*kt) + kt + Tt - ktp1 
    return ct, rt, Tt, wt


def CE(m, ktp2, ltp1, ktp1, lt, kt, ltm1, ztp1, zt):
    ct, rt, Tt, wt = bas(m, ktp1, kt, lt, zt)
    ctp1, rtp1, Ttp1, wtp1 = bas(m, ktp2, ktp1, ltp1, ztp1)
    rr_tp1 = (rtp1 - m.delta)*(1 - m.tau) + 1.0

    inter_ee = du_dc(m, ct, lt) - m.beta*du_dc(m, ctp1, ltp1)*rr_tp1
    intra_ee = du_dl(m, ct, lt) + du_dc(m, ct, lt)*wt*(1.0 - m.tau)

    return np.array([inter_ee, intra_ee])

In [32]:
def solve_ss(m):
    solve_me = lambda x: CE(m, x[0], x[1], x[0], x[1], x[0], x[1], 0.0, 0.0)
    sol = opt.root(solve_me, np.array([0.5, 0.85]))

    return sol.x

In [33]:
kbar, lbar = solve_ss(m)
cbar, rbar, Tbar, wbar = bas(m, kbar, kbar, lbar, 0.0)
Ybar = kbar ** m.alpha * np.exp(m.zbar) ** (1 - m.alpha)
ibar = m.delta * kbar

SS_vals = {'Kbar': kbar, 'Lbar': lbar, 'wbar': wbar,'Ybar': Ybar, 'ibar': ibar, 'cbar': cbar, 'rbar': rbar, 'Tbar': Tbar}

print(SS_vals)

{'Kbar': 327696.02954337833, 'Lbar': 316848.7933867982, 'wbar': 0.608133459704943, 'Ybar': 395.4153325870078, 'ibar': 321142.10895251075, 'cbar': 1.8125902396859601, 'rbar': 0.39200221252633694, 'Tbar': 0.09062951198429801}


In [ ]:
## Thank You!